In [ ]:
# Imports here
import numpy as np
import matplotlib.pyplot as plt
import os, random
import json
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision
from torchvision import datasets, transforms, models
from collections import OrderedDict
from PIL import Image
import time
import argparse

parser = argparse.ArgumentParser()

parser.add_argument('--dirdata', help = 'directory for data')
parser.add_argument('--dirsave', help = 'saving the data')
parser.add_argument('--epochs', default = 5, help = 'number of epochs')
parser.add_argument('--arch', default = 'alexnet')
parser.add_argument('--rate', default = 0.001, help = 'learning rate')
parser.add_argument('--gpu', dest = 'gpu', help = 'using gpu')
parser.add_argument('--hidden', dest = 'hidden', help = 'hidden layer')

args = parser.parse_args()
epochs = args.epochs
arch = args.arch
rate = args.rate
hidden = args.hidden
gpu = args.gpu

dirdata = './flowers'
training_dir = dirdata + '/train'
val_dir = dirdata + '/valid'
testing_dir = dirdata + '/test'

# Defining transforms for training, validation, and testing

training_transforms = transforms.Compose([
    transforms.RandomRotation(30),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([.485, .456, .406],
                         [.229, .224, .225])])

val_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([.485, .456, .406],
                         [.229, .224, .225])])

testing_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([.485, .456, .406],
                         [.229, .224, .225])])

# Load the dataset with ImageFolder
training_data = datasets.ImageFolder(training_dir, transform = training_transforms)
val_data = datasets.ImageFolder(val_dir, transform = val_transforms)
testing_data = datasets.ImageFolder(testing_dir, transform = testing_transforms)

# Using the image datasets and the transforms, define the dataloaders
trainingloader = torch.utils.data.DataLoader(training_data, batch_size = 64, shuffle = True)
valoader = torch.utils.data.DataLoader(val_data, batch_size = 64, shuffle = True)
testingloader = torch.utils.data.DataLoader(testing_data, batch_size = 64, shuffle = True)

imagesets = [training_data, val_data, testing_data]
dataloaders = [trainingloader, valoader, testingloader]

# Label mapping
with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Define device
if args.gpu and torch.cuda.is_available == True:
    device = torch.device('cuda')
else:
    device = torch.device('cpu')


# We will use alexnet as the default from 'Part 1'
if args.arch == 'alexnet':
    model = models.alexnet(pretrained = True) 
elif arch == 'vgg16':
    model = models.vgg16(pretrained = True) 
else:
    model = models.densenet121(pretrained = True)


# Build and train network
for param in model.parameters():
    param.requires_grad = False

classifier = nn.Sequential(
    OrderedDict([('fc1', nn.Linear(9216, 1024)),
                 ('relu1', nn.ReLU()),
                 ('drop1', nn.Dropout(p = 0.5)),
                 ('fc2', nn.Linear(1024, 512)),
                 ('relu2', nn.ReLU()),
                 ('drop2', nn.Dropout(p = 0.5)),
                 ('fc3', nn.Linear(512, 102)),
                 ('output', nn.LogSoftmax(dim = 1))]))
model.classifier = classifier

# Initialization of criterion and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr = args.rate)
model.to(device)

# Track the loss and accuracy on the validation set
def validation(model, valoader, criterion):
    model.to(device)
    
    loss = 0
    accuracy = 0
    
    for ii, (images, labels) in valoader:
        images, labels = images.to(device), labels.to(device)
        logps = model.forward(images)
        loss += criterion(logps, labels).item()
        
        ps = torch.exp(logps)
        equals = (labels.data == ps.max(dim = 1)[1])
        accuracy += equals.type(torch.FloatTensor).mean()
        
    return loss, accuracy

print_every = 28
steps = 0

# Running through epochs
for epoch in range(epochs):
    running_loss = 0
    for ii, (images, labels) in enumerate(trainingloader):
        steps += 1
        
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        
        logps = model.forward(images)
        loss = criterion(logps, labels)
        loss.backward()
        
        optimizer.step()
        running_loss += loss.item()
        
        if steps % print_every == 0:
            model.eval()
            
            with torch.no_grad():
                loss, accuracy = validation(model, valoader, criterion)
                
            print('Epoch {}/{}..'.format(epoch + 1, epochs),
                  'Training Loss: {:.3f}..'.format(running_loss / print_every),
                  'Valid Loss: {:.3f}..'.format(loss/len(valoader)),
                  'Valid Accuracy: {:.3f}%'.format(accuracy/len(valoader) * 100))
            
            running_loss = 0
            
            model.train()
            
# Checkpoint
model.to('cpu')
model.class_to_idx = training_data.class_to_idx

checkpoint = {'epochs': epochs,
              'mapping': model.class_to_idx,
              'arch': arch,
              'classifier': model.classifier,
              'state_dict': model.state_dict(),
              }

torch.save(checkpoint, 'checkpoint.pth')

In [ ]:
import argparse
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import torch
from torch import nn, optim
import os, random
import json
from collections import OrderedDict
import torchvision
from torchvision import datasets, models, transforms
import time
import torch.nn.functional as F
from torch.autograd import Variable

parser = argparse.ArgumentParser()

parser.add_argument('image_path', action = 'store')
parser.add_argument('checkpoint', action = 'store')
parser.add_argument('--topkm', default = 3, type = int)
parser.add_argument('--name_category', action = 'store')
parser.add_argument('--gpu', dest = 'gpu')

args = parser.parse_args()
image_path = args.image_path
checkpoint = args.checkpoint

# Write a function that loads a checkpoint and rebuilds the model
def modeload(checkpoint = 'checkpoint.pth'):
    checkpoint = torch.load(checkpoint)
    
    arch = checkpoint['arch']
    model.classifier = checkpoint['classifier']
    model.load_state_dict(checkpoint['state_dict'])
    model.epochs = checkpoint['epochs']
    model.class_to_idx = checkpoint['mapping']
        
    return model

# Image Preprocessing
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array'''
    imaj = Image.open(image)
    bonimaj = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean = [.485, .456, .406],
                             std = [.229, .224, .225])])
    
    imajforms = bonimaj(imaj)
    return imajforms
    
# Implement the code to predict the class from an image file
def predict(path, model, topkm):
    if gpu and torch.cuda.is_available() == 'gpu':
        model.to('cuda:0')
    
    # The image        
    image = process_image(path)
    image = image.unsqueeze_(0).float()
    if gpu == 'gpu':
        with torch.no_grad():
            image = image.cuda()
            output = model.forward(image)
    else:
        with torch.no_grad():
            output = model.forward(image)
            
    probability = F.softmax(output.data, dim = 1)
    
    return probability.topk(topkm)